# Variational AutoEncoder (MLP) Tensorflow
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
* https://www.tensorflow.org/get_started/embedding_viz
* https://www.youtube.com/watch?v=eBbEDRsCmv4
* https://www.youtube.com/watch?v=bbOFvxbMIV0
* https://www.youtube.com/watch?v=P78QYjWh5sM
* https://github.com/normanheckscher/mnist-tensorboard-embeddings
* http://projector.tensorflow.org/

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

SAVE_FOLDER='/tmp/vae_cnn'

# Delete directory if exist
if os.path.exists(SAVE_FOLDER):    
    os.system("rm -rf " + SAVE_FOLDER)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
from models import VAE_CNN
model = VAE_CNN(latent_size = 20)
model_in = model.input
model_out = model.output
model_out_flat = model.output_flat
z_mean = model.z_mean
z_stddev = model.z_stddev

### Define loss

In [3]:
with tf.name_scope("VAE_LOSS"):
    # Binary cross entropy
    generation_loss = -tf.reduce_sum(model_in * tf.log(1e-8 + model_out_flat) + (1-model_in) * tf.log(1e-8 + 1 - model_out_flat),1)
    
    # L2/L1 Loss (Works, but sometimes you my get a NaN)
    #generation_loss = tf.norm(model_in-model_out_flat, ord=1)
    #generation_loss = tf.nn.l2_loss(model_in-model_out_flat)
    
    # KL Loss
    latent_loss = 0.5 * tf.reduce_sum(tf.square(z_mean) + tf.square(z_stddev) - tf.log(tf.square(z_stddev)) - 1,1)
    
    # Merge the losses
    loss = tf.reduce_mean(generation_loss + latent_loss)

# Solver configuration
with tf.name_scope("Solver"):
    train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

### Build Graph

In [4]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

### Add some tensors to observe on Tensorboard

In [5]:
tf.summary.image("input_image", model.image_in, 4)
tf.summary.image("output_image", model_out, 4)
tf.summary.scalar("loss", loss)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(SAVE_FOLDER)
writer.add_graph(sess.graph)

# Create saver object
saver = tf.train.Saver()

### Train

In [6]:
num_epoch = 200
batch_size = 100
for epoch in range(num_epoch):
    for i in range(int(mnist.train.num_examples / batch_size)):
        # Get batch of 50 images
        batch = mnist.train.next_batch(50)

        # Dump summary
        if i % 5000 == 0:            
            # Other summaries
            s = sess.run(merged_summary, feed_dict={model_in:batch[0]})
            writer.add_summary(s,i) 
            
            # Save embedding (for PCA, TSNE)
            sess.run(model.assignment, feed_dict={model_in: batch[0]})
            saver.save(sess, os.path.join(SAVE_FOLDER, "model.ckpt"), i)


        # Train actually here (Also get loss value)    
        _, val_loss = sess.run((train_step, loss), feed_dict={model_in:batch[0]})

        #if i % 5000:
            #print('Generation loss: %d, latent loss: %d' % (np.mean(gen_loss), np.mean(lat_loss)))
    print('Epoch: %d/%d loss:%d' % (epoch, num_epoch, val_loss))

Epoch: 0/200 loss:225
Epoch: 1/200 loss:189
Epoch: 2/200 loss:154
Epoch: 3/200 loss:141
Epoch: 4/200 loss:145
Epoch: 5/200 loss:130
Epoch: 6/200 loss:123
Epoch: 7/200 loss:130
Epoch: 8/200 loss:127
Epoch: 9/200 loss:127
Epoch: 10/200 loss:121
Epoch: 11/200 loss:114
Epoch: 12/200 loss:112
Epoch: 13/200 loss:116
Epoch: 14/200 loss:120
Epoch: 15/200 loss:108
Epoch: 16/200 loss:101
Epoch: 17/200 loss:107
Epoch: 18/200 loss:106
Epoch: 19/200 loss:108
Epoch: 20/200 loss:106
Epoch: 21/200 loss:106
Epoch: 22/200 loss:117
Epoch: 23/200 loss:108
Epoch: 24/200 loss:105
Epoch: 25/200 loss:106
Epoch: 26/200 loss:107
Epoch: 27/200 loss:117
Epoch: 28/200 loss:109
Epoch: 29/200 loss:104
Epoch: 30/200 loss:109
Epoch: 31/200 loss:104
Epoch: 32/200 loss:100
Epoch: 33/200 loss:110
Epoch: 34/200 loss:106
Epoch: 35/200 loss:106
Epoch: 36/200 loss:104
Epoch: 37/200 loss:104
Epoch: 38/200 loss:107
Epoch: 39/200 loss:99
Epoch: 40/200 loss:104
Epoch: 41/200 loss:113
Epoch: 42/200 loss:106
Epoch: 43/200 loss:109